
discord pdf converter?


In [ ]:
const os = require('os')
const fs = require('fs')
const {updateInteraction} = importer.import('discord api')
const {savePdf} = importer.import('collect all bookmarks')
const {patchInteractionImageAttachment} = importer.import('update discord interaction with attachments')
const {doBrowserRequest} = importer.import('browser crawler tools')
const {
  safeurl,
  existingCache,
  storeCache,
  readCache,
} = importer.import('domain cache tools')

const TEMP_DIR = os.tmpdir();
let counter = 0

async function doDiscordPdf(interaction) {
  // overwrite old pdfs, just for API, should transfer in memory directly, but oh well
  const cache = existingCache(interaction.data.options[0].value, false)
  const filename = path.join(TEMP_DIR, 'temporary_discord_pdf_' + (counter % 4) + '.pdf')
  if(fs.existsSync(filename))
    fs.unlinkSync(filename)
  counter++
  await doBrowserRequest(interaction.data.options[0].value, 
    readCache.bind(null, cache),
    storeCache.bind(null, cache),
    savePdf.bind(null, filename))
  if(fs.existsSync(filename)) {
    return await patchInteractionImageAttachment(interaction.data.options[0].value, fs.readFileSync(filename), interaction.token, 'application/pdf')
  } else {
    return await updateInteraction('Page saving failed: ' + interaction.data.options[0].value, interaction.id, interaction.token)
  }
}

module.exports = {
  doDiscordPdf
}




discord notebook connector?


In [ ]:
const systemBuffer = importer.import('system usage png buffer')
const {patchInteractionImageAttachment} = importer.import('update discord interaction with attachments')
const {triggerTyping, createMessage, updateInteraction} = importer.import('disrcord api')
//const llama6 = importer.import('llm voice')
const summerizeArticle = importer.import('test article summarizer')

/*
async function doVoice(interaction) {
  const {llmVoice} = (await llama6)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmVoice('Convert the text to speech: ' + interaction.data.options[0].value)
  return await patchInteractionImageAttachment(interaction.data.options[0].value, Buffer.from(a1), interaction.token, 'audio/wav')
}
*/

async function doSystemUsage(interaction) {
  let buffer = await systemBuffer()
  await patchInteractionImageAttachment('system usage', buffer, interaction.token)
  let update
  let count = 24
  update = setInterval(async () => {
    count--
    buffer = await systemBuffer()
    await patchInteractionImageAttachment('system usage', buffer, interaction.token)
    if(count == 0) {
      clearInterval(update)
    }
  }, 4000)
}

async function doInterpret(interaction) {
  let code = await importer.interpret(interaction.data.options[0].value)
  await updateInteraction({
    content: code.language + ' code:\n```' + (code.language == 'node' ? 'javascript' : code.language) + '\n' + code.code.substring(0, 1800) + '\n```\n'
  }, interaction.id, interaction.token)
}

async function doSummary(interaction) {
  let summary = await summerizeArticle(interaction.data.options[0].value)
  await updateInteraction({
    content: 'Link: ' + interaction.data.options[0].value + '\nTLDR: ' + summary[1] + '\n\n' + summary[0]
  }, interaction.id, interaction.token)
}

module.exports = {
  doInterpret,
  doSystemUsage,
  //doVoice,
  doSummary,
}



discord llm connector?


In [ ]:
const {postMessageImageAttachment} = importer.import('create message image attachments')
const {triggerTyping, createMessage, updateInteraction} = importer.import('disrcord api')
const {Remarkable} = require('remarkable');
const md = new Remarkable({html: true, xhtmlOut: true, breaks: true});
const { safeurl } = importer.import('domain cache tools')
const selectModel = importer.import('select llm')
const askLlamaToWriteStory = importer.import('write creatively llm')
const askLlamaToWriteBusinessPlan = importer.import('business plan llm')
const askLlamaWriteEssay = importer.import('research paper llm')

async function doDistill(interaction, promptModel = 'DeepSeek') {

  if(typeof promptModel != 'function') {
    promptModel = await selectModel(promptModel)
  }

  await triggerTyping(interaction.channel_id)
  let q1 = 'Is this prompt looking for a long answer, answer Yes or No?\n' + interaction.data.options[0].value
  if((await promptModel(q1)).match(/no/gi)) {
    let a1 = await promptModel(interaction.data.options[0].value)
    return await updateInteraction(a1.substring(0, 1800), interaction.id, interaction.token)
  }
  await updateInteraction('This could take a while...', interaction.id, interaction.token)

  let q2 = 'Should I use the tool "essay" writer, "business plan", "creative story", or "default" prompter for this response?\n' + interaction.data.options[0].value
  let a2 = await promptModel(q2)
  if(a2.match(/essay/gi)) {
    let essay = await askLlamaWriteEssay(interaction.data.options[0].value, null, promptModel)
    return await postMessageImageAttachment(interaction.data.options[0].value, Buffer.from(essay), interaction.channel_id, 'text/html')
  } else if(a2.match(/business/gi)) {
    let plan = await askLlamaToWriteBusinessPlan(interaction.data.options[0].value, null, promptModel)
    // TODO: already writes to business-plans
    return await postMessageImageAttachment(interaction.data.options[0].value, Buffer.from(plan), interaction.channel_id, 'text/html')
  } else if(a2.match(/story/gi)) {
    let plan = await askLlamaToWriteStory(interaction.data.options[0].value, null, promptModel)
    // TODO: already writes to business-plans
    return await postMessageImageAttachment(interaction.data.options[0].value, Buffer.from(plan), interaction.channel_id, 'text/html')
  } else {
    let answer = await promptModel(interaction.data.options[0].value)
    const mdHtml = md.render(answer);
    return await postMessageImageAttachment(interaction.data.options[0].value, Buffer.from(mdHtml), interaction.channel_id, 'text/html')
  }
}

async function doReason(interaction) {
  return doDistill(interaction, 'Qwen')
}

async function doInstruct(interaction) {
  return doDistill(interaction, 'Llama')
}


module.exports = {
  doDistill,
  doReason,
  doInstruct,
}



discord writing llms?


In [ ]:
const {postMessageImageAttachment} = importer.import('create message image attachments')
const {triggerTyping, createMessage, updateInteraction} = importer.import('disrcord api')
const selectModel = importer.import('select llm')
const askLlamaToWriteStory = importer.import('write creatively llm')
const askLlamaToWriteBusinessPlan = importer.import('business plan llm')
const askLlamaWriteEssay = importer.import('research paper llm')


async function doEssay(interaction) {
  promptModel = await selectModel(interaction.data.options[1] ? interaction.data.options[1].value : 'Meta')

  await triggerTyping(interaction.channel_id)
  
  await updateInteraction('This could take a while...', interaction.id, interaction.token)

  let essay = await askLlamaWriteEssay(interaction.data.options[0].value, null, promptModel)

  return await postMessageImageAttachment(interaction.data.options[0].value, Buffer.from(essay), interaction.channel_id, 'text/html')
}

async function doBusiness(interaction) {
  promptModel = await selectModel(interaction.data.options[1] ? interaction.data.options[1].value : 'Meta')

  await triggerTyping(interaction.channel_id)
  
  await updateInteraction('This could take a while...', interaction.id, interaction.token)

  let essay = await askLlamaToWriteBusinessPlan(interaction.data.options[0].value, null, promptModel)

  return await postMessageImageAttachment(interaction.data.options[0].value, Buffer.from(essay), interaction.channel_id, 'text/html')
}

async function doStory(interaction) {
  promptModel = await selectModel(interaction.data.options[1] ? interaction.data.options[1].value : 'Meta')

  await triggerTyping(interaction.channel_id)
  
  await updateInteraction('This could take a while...', interaction.id, interaction.token)

  let essay = await askLlamaToWriteStory(interaction.data.options[0].value, null, promptModel)

  return await postMessageImageAttachment(interaction.data.options[0].value, Buffer.from(essay), interaction.channel_id, 'text/html')
}


module.exports = {
  doEssay,
  doBusiness,
  doStory,
}





discord mesh generator?


In [ ]:
const {postMessageImageAttachment} = importer.import('create message image attachments')
const llama = importer.import('llm mesh')
const {triggerTyping, createMessage, updateInteraction} = importer.import('disrcord api')

async function doMesh(interaction) {
  await triggerTyping(interaction.channel_id)
  await updateInteraction('This could take a while...', interaction.id, interaction.token)
  const llmMesh = (await llama)
  let a1 = await llmMesh('Create a model in OBJ format for: ' + interaction.data.options[0].value)
  let mesh = a1.substring(a1.indexOf('```obj')).replace(/```obj|```/gi, '')
  return await postMessageImageAttachment(interaction.data.options[0].value, Buffer.from(mesh), interaction.channel_id, 'model/obj')
}

module.exports = doMesh




discord llm interactions?


In [ ]:
const llama = importer.import('create llm session')
const {DEFAULT_APPLICATION} = importer.import('discord configuration')
const {triggerTyping, createMessage} = importer.import('disrcord api')

async function doInteraction(interaction) {
  const {llmPrompt} = (await llama)
  if(interaction.data.name == 'hello') {
    let a1 = await llmPrompt('hello')
    await updateInteraction({
      content: a1.substring(0, 1800)
    }, interaction.id, interaction.token)
  }
  else if (interaction.data.name == 'prompt') {
    let a1 = await llmPrompt(interaction.data.options[0].value + '\nmake it short.')
    await updateInteraction({
      content: a1.substring(0, 1800)
    }, interaction.id, interaction.token)
  }
}

async function doMention(interaction) {
  const {llmPrompt} = (await llama)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmPrompt(interaction.content.replace('@' + DEFAULT_APPLICATION) + '\nmake it short.')
  return await createMessage({
    content: a1.substring(0, 1800), 
    message_reference: { 
      message_id: interaction.id
    }
  }, interaction.channel_id)
}

async function doPrivate(interaction) {
  const {llmPrompt} = (await llama)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmPrompt(interaction.content)
  return await createMessage({
    content: a1.substring(0, 1800), 
    message_reference: { 
      message_id: interaction.id
    }
  }, interaction.channel_id)
}

module.exports = {
  doInteraction,
  doMention,
  doPrivate
}




sync discord llm tools?



export TOKENPATH=/Users/briancullinan/.credentials/discord-bot4.txt && export DEFAULT_APPLICATION=1335479586858733568 && npm run import -- "sync discord llm tools"




In [ ]:
const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')
const {interactionsCommands} = importer.import('discord gateway')
const {
  doInteraction, doMention, doPrivate
} = importer.import('discord llm interactions')
const {
  doDistill, doInstruct, doReason
} = importer.import('discord llm connector')
const {
  doEssay, doBusiness, doStory 
} = importer.import('discord writing llms')
const ALL_COMMANDS = [
  'hello',
  'prompt',
  'distill',
  'instruct',
  'reason',
  //'voice',
  'essay',
  'business',
  'story',
]

// bot commands using new API, same names as above but lower-case
async function syncChat(guildId = null) {
  var cmd, cmdDef
  var commandResult = await getCommands(guildId)
  var commands = commandResult.map(command => command.name)


  if(!commands.includes('hello')) {
    await registerCommand('hello', 'Check if Orbb is awake.', guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'hello')[0]
    //await updateCommand(cmd.name, cmd.description, cmd.id, guildId)
  }
  interactionsCommands['hello'] = doInteraction

  cmdDef = {
    'name': 'prompt',
    'description': 'Prompt an llm.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt the llm with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('prompt')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'prompt')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description 
      || cmdDef.options.length != cmd.options.length
      || JSON.stringify(cmdDef.options) != JSON.stringify(cmd.options.map(o => ({
        name: o.name,
        description: o.description,
        required: o.required,
        type: o.type
      })))) {
      debugger
      await updateCommand(cmdDef, null, cmd.id, guildId)
    }
  }
  interactionsCommands['prompt'] = doInteraction
  interactionsCommands['promptMention'] = doMention
  interactionsCommands['promptPrivate'] = doPrivate


  cmdDef = {
    'name': 'distill',
    'description': 'Distill with DeepSeek R1 8B.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt the DeepSeek R1 8B with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('distill')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'distill')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['distill'] = doDistill


  cmdDef = {
    'name': 'instruct',
    'description': 'Meta Instruct 70B.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt the Meta Instruct 70B with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('instruct')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'instruct')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['instruct'] = doInstruct

  cmdDef = {
    'name': 'reason',
    'description': 'Qwen 2.5 14B M1.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt Qwen reasoning LLM.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('reason')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'reason')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['reason'] = doReason

  cmdDef = {
    'name': 'essay',
    'description': 'Write an essay.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt LLM.',
        'required': true,
        'type': 3
      },
      {
        'name': 'model',
        'description': 'Model to use, Meta, DeepSeek, Llama, or Qwen.',
        'required': false,
        'type': 3,
        'choices': [
          {name: 'Meta', value: 'Meta'}, 
          {name: 'DeepSeek', value: 'DeepSeek'}, 
          {name: 'Llama', value: 'Llama'},
          {name: 'Qwen', value: 'Qwen'},
          {name: 'Code', value: 'Code'},
        ]
      }
    ]
  }
  if(!commands.includes('essay')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'essay')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['essay'] = doEssay

  cmdDef = {
    'name': 'business',
    'description': 'Write an business plan.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt LLM.',
        'required': true,
        'type': 3
      },
      {
        'name': 'model',
        'description': 'Model to use, Meta, DeepSeek, Llama, or Qwen.',
        'required': false,
        'type': 3,
        'choices': [
          {name: 'Meta', value: 'Meta'}, 
          {name: 'DeepSeek', value: 'DeepSeek'}, 
          {name: 'Llama', value: 'Llama'},
          {name: 'Qwen', value: 'Qwen'},
          {name: 'Code', value: 'Code'},
        ]
      }
    ]
  }
  if(!commands.includes('business')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'business')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['business'] = doBusiness

  cmdDef = {
    'name': 'story',
    'description': 'Write an story.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt LLM.',
        'required': true,
        'type': 3
      },
      {
        'name': 'model',
        'description': 'Model to use, Meta, DeepSeek, Llama, or Qwen.',
        'required': false,
        'type': 3,
        'choices': [
          {name: 'Meta', value: 'Meta'}, 
          {name: 'DeepSeek', value: 'DeepSeek'}, 
          {name: 'Llama', value: 'Llama'},
          {name: 'Qwen', value: 'Qwen'},
          {name: 'Code', value: 'Code'},
        ]
      }
    ]
  }
  if(!commands.includes('story')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'story')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['story'] = doStory

  /*
  cmdDef = {
    'name': 'voice',
    'description': 'OuteTTS 0.3 1B/Llasa 3B gguf.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to convert to audio.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('voice')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'voice')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['voice'] = doVoice
  */


  var toRemove = commandResult.filter(c => !ALL_COMMANDS.includes(c.name))
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }

  return await getCommands()
}

module.exports = syncChat



sync deceptive chat?


export TOKENPATH=/Users/briancullinan/.credentials/discord-bot3.txt && export DEFAULT_APPLICATION=1335472379232780401 && npm run import -- "sync deceptive chat"


In [ ]:
const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')
const {interactionsCommands} = importer.import('discord gateway')
const llama = importer.import('llm deceive')
const {triggerTyping, createMessage, updateInteraction} = importer.import('disrcord api')
const {DEFAULT_APPLICATION} = importer.import('discord configuration')
const argueLlama = importer.import('argue with multiple llms')

const ALL_COMMANDS = [
  'deceive',
  'argue',
]

async function doDeceive(interaction) {
  const llmDeceive = (await llama)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmDeceive('Give the most wrong or incorrect response to, be as deceptive as possible:\n' + interaction.data.options[0].value)
  return await updateInteraction(a1.substring(0, 1800), interaction.id, interaction.token)
}

async function doArgument(interaction) {
  let count = 0
  return Promise.resolve(argueLlama(interaction.data.options[0].value, async (prompt, answer) => {
    if(count == 0) {
      // don't repost original prompt
      await updateInteraction(answer.substring(0, 1800), interaction.id, interaction.token)
    } else {
      //await createMessage(prompt.substring(0, 1800), interaction.channel_id)
      await createMessage(answer.substring(0, 1800), interaction.channel_id)
    }
    count++
  }))
}

async function doMention(interaction) {
  const llmDeceive = (await llama)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmDeceive(interaction.content.replace('@' + DEFAULT_APPLICATION) + '\nBe as deceptive as possible and make it short.')
  return await createMessage({
    content: a1.substring(0, 1800), 
    message_reference: { 
      message_id: interaction.id
    }
  }, interaction.channel_id)
}

async function doPrivate(interaction) {
  const llmDeceive = (await llama)
  await triggerTyping(interaction.channel_id)
  let a1 = await llmDeceive('Deceive me with the opposite of:\n' + interaction.content)
  return await createMessage({
    content: a1.substring(0, 1800), 
    message_reference: { 
      message_id: interaction.id
    }
  }, interaction.channel_id)
}


// bot commands using new API, same names as above but lower-case
async function syncTools(guildId = null) {
  var cmd, cmdDef
  var commandResult = await getCommands(guildId)
  var commands = commandResult.map(command => command.name)


  cmdDef = {
    'name': 'deceive',
    'description': 'DeepSeek 7B Chat.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt deceptive LLM with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('deceive')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'deceive')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['deceive'] = doDeceive
  interactionsCommands['promptMention'] = doMention
  interactionsCommands['promptPrivate'] = doPrivate


  cmdDef = {
    'name': 'argue',
    'description': 'Start an argument between deceptive DeepSeek 7B Chat and Meta Llama 8B.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt argumentative LLM with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('argue')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'argue')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['argue'] = doArgument


  var toRemove = commandResult.filter(c => !ALL_COMMANDS.includes(c.name))
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }

  return await getCommands()
}

module.exports = syncTools



discord handy tools?


export TOKENPATH=/Users/briancullinan/.credentials/discord-bot5.txt && export DEFAULT_APPLICATION=1335483072602177626 && npm run import -- "discord handy tools"



In [ ]:

const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')
const {interactionsCommands} = importer.import('discord gateway')
const {doDiscordPdf} = importer.import('discord pdf converter')
const {
  doInterpret, doSystemUsage, doSummary
} = importer.import('discord notebook connector')
const doMesh = importer.import('discord mesh generator')

const ALL_COMMANDS = [
  'pdf',
  'interpret',
  'usage',
  'mesh',
  'summary',
]

// bot commands using new API, same names as above but lower-case
async function syncTools(guildId = null) {
  var cmd, cmdDef
  var commandResult = await getCommands(guildId)
  var commands = commandResult.map(command => command.name)


  cmdDef = {
    'name': 'pdf',
    'description': 'Convert a website to PDF.',
    'options': [
      {
        'name': 'address',
        'description': 'Http address to convert.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('pdf')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'pdf')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['pdf'] = doDiscordPdf


  cmdDef = {
    'name': 'interpret',
    'description': 'Interpret what a tool will do.',
    'options': [
      {
        'name': 'search',
        'description': 'Search function for the tool name.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('interpret')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'interpret')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['interpret'] = doInterpret


  cmdDef = {
    'name': 'usage',
    'description': 'View system usage.',
    'options': []
  }
  if(!commands.includes('usage')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'usage')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['usage'] = doSystemUsage



  cmdDef = {
    'name': 'summary',
    'description': 'Summarize an article with LLM.',
    'options': [
      {
        'name': 'url',
        'description': 'Link address to summarize.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('summary')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'summary')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['summary'] = doSummary



  cmdDef = {
    'name': 'mesh',
    'description': 'LLaMa-Mesh gguf.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt mesh generator with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('mesh')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'mesh')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['mesh'] = doMesh



  var toRemove = commandResult.filter(c => !ALL_COMMANDS.includes(c.name))
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }

  return await getCommands()
}

module.exports = syncTools




create discord message with attachments?


In [ ]:
const { safeurl } = importer.import('domain cache tools')
const mime = require('mime-types');
const {requestAuthQ} = importer.import('discord request')


async function postMessageImageAttachment(prompt, buff, channelId, content = 'image/png') {
  return await requestAuthQ({
    headers: {
      'Content-Type': 'multipart/form-data; boundary=boundary123'
    },
    method: 'POST',
    url: `channels/${channelId}/messages`,
    body: `--boundary123
Content-Disposition: form-data; name="payload_json"
Content-Type: application/json

${JSON.stringify({
  content: 'Your ' + (content.includes('image') ? 'image' : content) + ' delivered: *' + prompt + '*',
  attachments: [{
    "id": 0,
    "description": prompt,
    "filename": safeurl(prompt) + '.' + mime.extension(content)
  }],
})}
--boundary123
Content-Disposition: form-data; name="files[0]"; filename="${safeurl(prompt)}.${mime.extension(content)}"
Content-Transfer-Encoding: base64
Content-Type: ${content}

${buff.toString('base64')}
--boundary123--`
  })
}


module.exports = {
  postMessageImageAttachment
}
